In [1]:
import torch
from transformers import BertTokenizer, BertTokenizerFast
from transformers import AutoTokenizer

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
version = "bert-base-uncased"
sequence = "The quick brown fox jumps over the lazy dog."
max_length = 20

# BertTokenizer

In [4]:
tokenizer: BertTokenizer = BertTokenizer.from_pretrained(version)
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

## special ids and tokens

In [5]:
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens)

[100, 102, 0, 101, 103]
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [6]:
tokenizer.ids_to_tokens

OrderedDict([(0, '[PAD]'),
             (1, '[unused0]'),
             (2, '[unused1]'),
             (3, '[unused2]'),
             (4, '[unused3]'),
             (5, '[unused4]'),
             (6, '[unused5]'),
             (7, '[unused6]'),
             (8, '[unused7]'),
             (9, '[unused8]'),
             (10, '[unused9]'),
             (11, '[unused10]'),
             (12, '[unused11]'),
             (13, '[unused12]'),
             (14, '[unused13]'),
             (15, '[unused14]'),
             (16, '[unused15]'),
             (17, '[unused16]'),
             (18, '[unused17]'),
             (19, '[unused18]'),
             (20, '[unused19]'),
             (21, '[unused20]'),
             (22, '[unused21]'),
             (23, '[unused22]'),
             (24, '[unused23]'),
             (25, '[unused24]'),
             (26, '[unused25]'),
             (27, '[unused26]'),
             (28, '[unused27]'),
             (29, '[unused28]'),
             (30, '[unused29]'),
  

## tokenize(sequence) 对文本进行分词

In [7]:
tokens = tokenizer.tokenize(sequence)
tokens

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.']

## convert_tokens_to_ids 将分词后的token映射为数字

In [8]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[1996, 4248, 2829, 4419, 14523, 2058, 1996, 13971, 3899, 1012]

## encode = tokenize + convert_tokens_to_ids

In [9]:
print(tokenizer.encode(sequence, add_special_tokens=False))
ids = tokenizer.encode(sequence, add_special_tokens=True)
ids

[1996, 4248, 2829, 4419, 14523, 2058, 1996, 13971, 3899, 1012]


[101, 1996, 4248, 2829, 4419, 14523, 2058, 1996, 13971, 3899, 1012, 102]

In [10]:
tokenizer.convert_ids_to_tokens([101, 102])

['[CLS]', '[SEP]']

## batch_encode_plus = batch encode

In [11]:
tokenizer.batch_encode_plus([sequence], add_special_tokens=True, padding=True)

{'input_ids': [[101, 1996, 4248, 2829, 4419, 14523, 2058, 1996, 13971, 3899, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

## convert_ids_to_tokens 将数字映射为token

In [12]:
print(tokenizer.convert_ids_to_tokens(ids))
tokens = tokenizer.convert_ids_to_tokens(ids, skip_special_tokens=True)
tokens

['[CLS]', 'the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.', '[SEP]']


['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.']

## convert_tokens_to_string 将token转换为string

In [13]:
tokenizer.convert_tokens_to_string(tokens)

'the quick brown fox jumps over the lazy dog .'

## decode = convert_ids_to_tokens + convert_tokens_to_string

In [14]:
print(tokenizer.decode(ids))
print(tokenizer.decode(ids, skip_special_tokens=True))

[CLS] the quick brown fox jumps over the lazy dog. [SEP]
the quick brown fox jumps over the lazy dog.


## batch_decode = batch decode

In [15]:
print(tokenizer.batch_decode([ids]))
print(tokenizer.batch_decode([ids], skip_special_tokens=True))

['[CLS] the quick brown fox jumps over the lazy dog. [SEP]']
['the quick brown fox jumps over the lazy dog.']


## tokenizer([sequence])

In [16]:
inputs = tokenizer(
    sequence,  # 单个句子
    truncation=True,  # 超出max_length截断处理
    padding=True,  # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,          # 最长长度,不设置默认为模型最大长度
    add_special_tokens=True,  # text添加特殊key
    return_length=True,  # 返回有效长度
    return_attention_mask=True,  # 返回attention_mask
    return_overflowing_tokens=False,  # 返回所有的文本片段（由于文本比较长，默认情况下超过预设截断长度的token会被丢失。如果设置了return_overflowing_tokens=True则会返回所有的token片段）。
    return_tensors="pt",  # 返回数据格式 np pt tf jax
).to(device)

print(inputs.keys())
print(inputs["input_ids"])  # 对应文字id
print(inputs["attention_mask"])  # 对应是否是文字
print(inputs["length"])  # 对应总长度长度

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'length'])
tensor([[  101,  1996,  4248,  2829,  4419, 14523,  2058,  1996, 13971,  3899,
          1012,   102]], device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
tensor([12], device='cuda:0')


In [17]:
inputs = tokenizer(
    [sequence] * 2,  # 句子batch
    truncation=True,  # 超出max_length截断处理
    padding=True,  # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,          # 最长长度,不设置默认为模型最大长度
    return_length=True,  # 返回有效长度
    return_attention_mask=True,  # 返回attention_mask
    return_overflowing_tokens=False,  # 返回所有的文本片段（由于文本比较长，默认情况下超过预设截断长度的token会被丢失。如果设置了return_overflowing_tokens=True则会返回所有的token片段）。
    return_tensors="pt",  # 返回数据格式 np pt tf jax
).to(device)

print(inputs.keys())
print(inputs["input_ids"])
print(inputs["attention_mask"])  # 对应是否是文字
print(inputs["length"])  # 对应有效文字长度

dict_keys(['input_ids', 'token_type_ids', 'length', 'attention_mask'])
tensor([[  101,  1996,  4248,  2829,  4419, 14523,  2058,  1996, 13971,  3899,
          1012,   102],
        [  101,  1996,  4248,  2829,  4419, 14523,  2058,  1996, 13971,  3899,
          1012,   102]], device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
tensor([12, 12], device='cuda:0')


# BertTokenizerFast

In [18]:
tokenizer: BertTokenizerFast = BertTokenizerFast.from_pretrained(version)
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [19]:
inputs = tokenizer(
    [sequence] * 2,  # 句子batch
    truncation=True,  # 超出max_length截断处理
    padding=True,  # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,          # 最长长度,不设置默认为模型最大长度
    add_special_tokens=True,  # text添加特殊key
    return_length=True,  # 返回有效长度
    return_attention_mask=True,  # 返回attention_mask
    return_overflowing_tokens=False,  # 返回所有的文本片段（由于文本比较长，默认情况下超过预设截断长度的token会被丢失。如果设置了return_overflowing_tokens=True则会返回所有的token片段）。
    return_tensors="pt",  # 返回数据格式 np pt tf jax
).to(device)

print(inputs.keys())
print(inputs["input_ids"])
print(inputs["attention_mask"])  # 对应是否是文字
print(inputs["length"])  # 返回文字长度时最长长度,结果不对

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'length'])
tensor([[  101,  1996,  4248,  2829,  4419, 14523,  2058,  1996, 13971,  3899,
          1012,   102],
        [  101,  1996,  4248,  2829,  4419, 14523,  2058,  1996, 13971,  3899,
          1012,   102]], device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
tensor([12, 12], device='cuda:0')


# AutoTokenizer 通用封装，根据载入预训练模型来自适应

https://huggingface.co/docs/transformers/main/autoclass_tutorial#autotokenizer

In [20]:
#                                                               默认为Fast,可以不用
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(version, use_fast=False)
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [21]:
inputs = tokenizer(
    [sequence] * 2,  # 句子batch
    truncation=True,  # 超出max_length截断处理
    padding=True,  # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,          # 最长长度,不设置默认为模型最大长度
    add_special_tokens=True,  # text添加特殊key
    return_length=True,  # 返回有效长度
    return_attention_mask=True,  # 返回attention_mask
    return_overflowing_tokens=False,  # 返回所有的文本片段（由于文本比较长，默认情况下超过预设截断长度的token会被丢失。如果设置了return_overflowing_tokens=True则会返回所有的token片段）。
    return_tensors="pt",  # 返回数据格式 np pt tf jax
).to(device)

print(inputs.keys())
print(inputs["input_ids"])
print(inputs["attention_mask"])  # 对应是否是文字
print(inputs["length"])  # 返回文字最长长度

dict_keys(['input_ids', 'token_type_ids', 'length', 'attention_mask'])
tensor([[  101,  1996,  4248,  2829,  4419, 14523,  2058,  1996, 13971,  3899,
          1012,   102],
        [  101,  1996,  4248,  2829,  4419, 14523,  2058,  1996, 13971,  3899,
          1012,   102]], device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
tensor([12, 12], device='cuda:0')
